---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
print("Dependencies:-")
print("Pandas :",pd.__version__)
print("Numpy  :",np.__version__)
print("Scipy  :","1.0.0")

Dependencies:-
Pandas : 0.19.2
Numpy  : 1.12.1
Scipy  : 1.0.0


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv)
, ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

# Filtering gdplev.xls
gdplev_columns = ['Unnamed: 4','GDP in billions of chained 2009 dollars.1']
gdplev_df = pd.read_excel('gdplev.xls',skiprows=5).iloc[214:].reset_index()[gdplev_columns].rename(columns = {'Unnamed: 4':'Quaters','GDP in billions of chained 2009 dollars.1':'GDP'})

# Filtering university_towns.txt
txt_data = open('university_towns.txt')
filtered_data = []
for line in txt_data.readlines():
    line = line.replace("€“","") # found those from debug dataset : https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/QRjk3bKzEee5Ew6CaivGLg
    if "[edit]" in line:
        state = line[:line.find('[')]
    else:
        region = line[:line.find(' (')]
        filtered_data.append((state,region))
university_towns_df = pd.DataFrame(filtered_data,columns=["State", "RegionName"])

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return university_towns_df.copy()

In [4]:
get_list_of_university_towns().T.head(3)

,0,1,2,3,4,5,6,7,8,9,...,507,508,509,510,511,512,513,514,515,516
State,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alaska,Arizona,...,Wisconsin,Wisconsin,Wisconsin,Wisconsin,Wisconsin,Wisconsin,Wisconsin,Wisconsin,Wisconsin,Wyoming
RegionName,Auburn,Florence,Jacksonville,Livingston,Montevallo,Troy,Tuscaloosa,Tuskegee,Fairbanks,Flagstaff,...,Madison,Menomonie,Milwaukee,Oshkosh,Platteville,River Falls,Stevens Point,Waukesha,Whitewater,Laramie


In [5]:
get_gdp = lambda x : gdplev_df.iloc[x]['GDP']
gdplev_df['Diff'] = 0
gdplev_df['Small_Diff'] = 0

for index in range(1 , gdplev_df.shape[0]-1):
    previous_year , present_year , next_year = get_gdp(index-1) , get_gdp(index) , get_gdp(index+1)
    gdplev_df['Diff'].iloc[index] = present_year - previous_year
    
    if (previous_year > present_year):
        gdplev_df['Small_Diff'].iloc[index] = -1
    elif(previous_year < present_year):
        gdplev_df['Small_Diff'].iloc[index] = 1

D:\Anaconda\envs\learnpython\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
def get_recession_details():
    df = gdplev_df.copy()
    get_gdp = lambda x : df.iloc[x]['Small_Diff']
    
    recession = []

    for index in range(1 , df.shape[0]-2):
        present_gdp , next_gdp = get_gdp(index), get_gdp(index+1)
        if (present_gdp == -1 and next_gdp == -1):
            
            for sub_index in range(index+2,df.shape[0]-3):
                sub_present_gdp , sub_next_gdp = get_gdp(sub_index), get_gdp(sub_index+1)
                
                if (sub_present_gdp == 1 and sub_next_gdp == 1):
                    recession = [ index , sub_index+1 ]
                    break

            if len(recession) == 2:
                break
    return df.iloc[recession[0]:recession[1]+1]

In [7]:
get_recession_details()

,Quaters,GDP,Diff,Small_Diff
34,2008q3,14891.6,-71.8,-1
35,2008q4,14577.0,-314.6,-1
36,2009q1,14375.0,-202.0,-1
37,2009q2,14355.6,-19.4,-1
38,2009q3,14402.5,46.9,1
39,2009q4,14541.9,139.4,1


In [8]:
def get_recession_start():
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return get_recession_details().iloc[0]['Quaters']

In [9]:
get_recession_start()

'2008q3'

In [10]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return get_recession_details().iloc[-1]['Quaters']

In [11]:
get_recession_end()

'2009q4'

In [12]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    return get_recession_details().sort_values(by="GDP").iloc[0]['Quaters']

In [13]:
get_recession_bottom()

'2009q2'

### Housing Data problem

In [14]:
def convert_housing_data_to_quarters():
    '''
    Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''  
    datadf = pd.read_csv('City_Zhvi_AllHomes.csv')
    datadf = datadf.drop(datadf[[0]+list(range(3,51))] , axis = 1)
    datadf['State'] = datadf.apply(lambda x: states[x["State"]] , axis=1 )
    
    finaldf = datadf.copy()
    #data_df.index += 1
    
    finaldf = finaldf[["State","RegionName"]].copy()
    
    for (index,year) in enumerate(range(2000,2016)):
        finaldf[str(year)+"q1"] = datadf[[str(year)+"-01",str(year)+"-02",str(year)+"-03"]].mean(axis=1)
        finaldf[str(year)+"q2"] = datadf[[str(year)+"-04",str(year)+"-05",str(year)+"-06"]].mean(axis=1)
        finaldf[str(year)+"q3"] = datadf[[str(year)+"-07",str(year)+"-08",str(year)+"-09"]].mean(axis=1)
        finaldf[str(year)+"q4"] = datadf[[str(year)+"-10",str(year)+"-11",str(year)+"-12"]].mean(axis=1)
        
    finaldf["2016q1"] = datadf[["2016-01","2016-02","2016-03"]].mean(axis=1)
    finaldf["2016q2"] = datadf[["2016-04","2016-05","2016-06"]].mean(axis=1)
    finaldf["2016q3"] = datadf[["2016-07","2016-08"]].mean(axis=1)
    finaldf = finaldf.set_index(['State','RegionName'])
    return finaldf.copy()

In [15]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()

    housing = convert_housing_data_to_quarters()
    housing = housing[rec_start].div(housing[rec_bottom]) # price_ratio=quarter_before_recession/recession_bottom

    uni = get_list_of_university_towns()
    spam = list(zip(uni["State"],uni["RegionName"]))
    filtr = housing.index.isin(spam)
    uni_rec = housing[filtr].dropna()
    nonuni_rec = housing[filtr != True].dropna()

    p_value=ttest_ind(list(uni_rec),list(nonuni_rec))[1]

    # p-test
    status = True if p_value < 0.01 else False
    category = "university town" if (uni_rec.mean() < nonuni_rec.mean()) else "non-university town"
    return((status,p_value,category))

In [16]:
run_ttest()

(True, 0.0054964273536938866, 'university town')

### Make sure you have 269 university towns and 10461 non-university towns in the final analysis.

In [88]:
rec_start = get_recession_start()
rec_bottom = get_recession_bottom()

housing = convert_housing_data_to_quarters()
housing = housing[rec_start].div(housing[rec_bottom]) # price_ratio=quarter_before_recession/recession_bottom

uni = get_list_of_university_towns()
spam = list(zip(uni["State"],uni["RegionName"]))
filtr = housing.index.isin(spam)
uni_rec = housing[filtr].dropna()
nonuni_rec = housing[filtr != True].dropna()

p_value=ttest_ind(list(uni_rec),list(nonuni_rec))[1]

# p-test
status = True if p_value < 0.01 else False
category = "university town" if (uni_rec.mean() < nonuni_rec.mean()) else "non-university town"
print((status,p_value,category))

(True, 0.0054964273536938866, 'university town')
